<img src='../img/EU-Copernicus-EUM_banner_logo.png' align='right' width='50%'></img>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_2_OLCI_file_structure.ipynb">OLCI file structure >></a>

Copernicus Marine Training Service <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-info">
<h1>Learn OLCI: Introductory</h1></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **None**

It is recommended to go through these modules before you start with this module.
</div>

<hr>

## OLCI data access 

### Learning outcomes

### Outline

### Downloaded products
S3A_OL_1_EFR____20210717T101015_20210717T101315_20210718T145224_0179_074_122_1980_MAR_O_NT_002.SEN3
S3A_OL_2_WFR____20210717T101015_20210717T101315_20210718T221347_0179_074_122_1980_MAR_O_NT_003.SEN3
S3A_OL_2_WRR____20210717T095732_20210717T104152_20210718T152419_2660_074_122______MAR_O_NT_003.SEN3

<hr>

In [ ]:
import shutil
import os
import json
import zipfile
import eumdac #eumetsat/data-store
import hda #ecmwf/wekeo

## Via the Data Store (using the eumdac library) -- what about CLI?

In [ ]:
# load your credentials and generate a token
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

In [ ]:
# Retrieve the required collection objects from DataStore
collectionID = 'EO:EUM:DAT:0407'
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection(collectionID)

In [ ]:
# Find the required product in the collection
selected_product = datastore.get_product(
    product_id='S3B_OL_2_WFR____20220107T100135_20220107T100435_20220107T112502_0179_061_150_4500_MAR_O_NR_003.SEN3',
    collection_id=collectionID)

In [ ]:
# Create the download directory
os.makedirs(os.path.join(os.getcwd(),'products'), exist_ok = True)

In [ ]:
# Download the product
with selected_product.open() as fsrc, \
        open(os.path.join(os.getcwd(), 'products', fsrc.name), mode='wb') as fdst:
    shutil.copyfileobj(fsrc, fdst)
    print(f'Download of product {selected_product} finished.')

In [ ]:
# Unzip **only** SAFE format files and clear-up latent zip file
with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.startswith(str(selected_product)):
            zip_ref.extract(file, os.path.join(os.getcwd(),'products'))
os.remove(fdst.name)

## Via WEkEO (using the hda)
https://github.com/ecmwf/hda

<hr>

<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>